In [ ]:
!pip install -q unsloth
!pip uninstall -q unsloth -y && pip install -q --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

In [ ]:
import unsloth
import os
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get('hf_token')

if HF_TOKEN:
    try:
        login(HF_TOKEN)
        print('Login Hugging Face successful')
    except Exception as e:
        print(f'{str[e]}')

## Define Model name, Data and Personal HF repo


In [ ]:
model_name = 'unsloth/Qwen3-4B-unsloth-bnb-4bit'
repo_name = 'VyDat/Qween3-4B-bnb-4bit'

data = '5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca'

## Load model


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    use_gradient_checkpointing='unsloth',
)

## Create PEFT model


In [ ]:
model = FastLanguageModel.get_peft_model(
    model=model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout= 0.05,
    use_gradient_checkpointing='unsloth',
    bias= 'none',
    random_state=3407
)

## Format Dataset


In [ ]:
def format_data(data_path):
    try:
        dataset = load_dataset(data_path, split='train')
    except Exception as e:
        print(f'ERROR loading data {str(e)}')
        
    def format_func(data_set):
        texts = []

        for conversations in data_set['conversations']:
            fixed_msg = []

            for i, msg in enumerate(conversations):
                if not isinstance(msg, dict) or 'from' not in msg:
                    print(f"Invalid message at turn {i}")
                    continue

                role = 'assistant' if msg['from'] == 'gpt' else 'user'
                fixed_msg.append({
                    'role': role,
                    'content': msg['value']
                })

            if fixed_msg:
                text = tokenizer.apply_chat_template(
                    fixed_msg,
                    tokenize=False,
                    add_generation_prompt=False
                )
                texts.append(text)

        return {'text': texts}
    
    formated_dataset = dataset.map(
        format_func,
        batched=True,
        remove_columns=dataset.column_names,
        num_proc=4
    )
    return formated_dataset

formated_dataset = format_data(data)
formated_dataset[0]['text']

In [ ]:
split_data = formated_dataset.train_test_split(test_size=0.1, seed=42)
train_data = split_data['train']
eval_data = split_data['test']
print(f'Train: {len(train_data)}, Eval: {len(eval_data)}')

In [ ]:
arg = TrainingArguments(
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    num_train_epochs=2,
    learning_rate=3e-4,
    weight_decay=1e-5,
    fp16= not torch.cuda.is_bf16_supported(),
    bf16= torch.cuda.is_bf16_supported(),
    logging_steps=100,
    optim='adamw_8bit',
    lr_scheduler_type='linear',
    seed=42,
    output_dir='outputs',
    save_steps=200,
    save_strategy='steps',
    eval_steps=200,
    eval_strategy='steps',
    save_total_limit=3,
    report_to='none',
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=arg,
    train_dataset=train_data,
    eval_dataset=eval_data,
    dataset_text_field='text',
    max_seq_length = 2048,
    dataset_num_proc=4,
    packing=True
)

In [ ]:
print("Start Trainning.....")
torch.cuda.empty_cache()

trainer_stats = trainer.train()

print(f"Trainning successfull, Total Loss: {trainer_stats.training_loss}")

## Merge LoRA


In [ ]:
model = model.merge_and_upload()

model.save_pretrained(repo_name)
tokenizer.save_pretrained(repo_name)

try:
    model.push_to_hub(repo_name, use_temp_dir=True)
    tokenizer.push_to_hub(repo_name, use_temp_dir=True)
    print('Push to hub successful')
    print(f'Model: https://huggingface.co/{repo_name}')
except Exception as e:
    print(f'{str(e)}')